In [2]:
import time
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt 
import wavio
import scipy.io.wavfile

In [3]:
RATE = 48000
duration = 0.3
frequency = 220.0
fm = 0.25

In [4]:
x = np.linspace(0, 2 * duration * np.pi, int(RATE * duration))


In [5]:
fade_amount = 8000
detune = 5.0

fade = np.linspace(1, 0, fade_amount)

In [6]:
def sine_wave(f, detune=0.0):
    y = np.sin((f + detune) * x + ramp *
            np.sin(((f + detune) * 0.5) * x + (
                np.sin(((f + detune) * fm) * x) * 0.5)))
    
    return y


In [7]:
scale_ramp = []
index = 0
while index <= 200:
    scale_ramp.append(index / 100)
    index += 1
    

In [8]:
scale_roll = []
index = 0
while index <= 4000:
    scale_roll.append(index)
    index += 50

In [9]:
len(scale_roll)

81

In [10]:
waves = []


In [29]:
for ramp_amount in scale_ramp:
    ramp = np.logspace(1, 0, np.size(x), base=10) * ramp_amount
    
    waveform_mod = sine_wave(frequency)
    waveform = sine_wave(frequency)
    waveform_detune = sine_wave(frequency, detune)
    
    waveform = ((waveform + waveform_detune) * 
                (waveform_mod / 2 + 0.5)) * 0.1
    
    waveform[-fade_amount:] *= fade
    waveform = np.int16(waveform * 32767)
    waveform2 = np.roll(waveform, 10, axis=None)
    waveform3 = np.vstack((waveform2, waveform)).T
    waves.append(waveform3.copy(order='C'))

In [30]:
wave1 = waves[0]

In [31]:
total_wave = wave1
for w in waves:
    total_wave = np.concatenate((total_wave, w))
    

In [32]:
len(total_wave) / RATE

120.9

In [33]:
attenuated = total_wave * 0.3

In [34]:
try:
    sd.play(attenuated, RATE)

except KeyboardInterrupt:
    sd.stop()

In [35]:
sd.stop()
%matplotlib 

Using matplotlib backend: TkAgg


In [42]:
fix, ax = plt.subplots()
ax.plot(attenuated[:-RATE*60], linewidth=1.5)
plt.show()